In [3]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Large,MobileNetV3Small
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
import os
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [6]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50 # Tingkatkan epoch, EarlyStopping akan menangani jika terlalu banyak
NUM_CLASSES = len(os.listdir('dataset/train')) # Pastikan path ini benar

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2, # Tambahan augmentasi ringan
    height_shift_range=0.2, # Tambahan augmentasi ringan
    shear_range=0.2, # Tambahan augmentasi ringan
    horizontal_flip=True,
    fill_mode='nearest' # Mode untuk mengisi piksel baru setelah augmentasi
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load base model
base_model = MobileNetV3Large(weights='imagenet', include_top=False, input_shape=(224, 224, 3), include_preprocessing=False)

# --- Fine-tuning strategy ---
# Freeze sebagian besar layer awal, dan unfreeze beberapa layer terakhir
base_model.trainable = True

# Tentukan berapa banyak layer dari akhir yang ingin di-fine-tune
# Ini adalah hyperparameter, Anda mungkin perlu bereksperimen
# Mari kita coba unfreeze 20 layer terakhir dari MobileNetV3Large
# Anda bisa mengecek jumlah layer total dengan: print(len(base_model.layers))
fine_tune_at_layer_count = 20 # Unfreeze N layer terakhir
if len(base_model.layers) > fine_tune_at_layer_count:
    for layer in base_model.layers[:-fine_tune_at_layer_count]: # Bekukan semua KECUALI N terakhir
        layer.trainable = False
else: # Jika model lebih pendek dari N, unfreeze semua
    print(f"Peringatan: Model memiliki {len(base_model.layers)} layer, kurang dari {fine_tune_at_layer_count} yang diminta untuk fine-tune. Semua layer base model akan dilatih.")
    for layer in base_model.layers:
        layer.trainable = True
# --------------------------

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Anda bisa mencoba menambah layer Dense di sini jika diperlukan
# x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x) # Sesuaikan dropout rate jika perlu (misal 0.4 atau 0.5)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model dengan learning rate yang lebih rendah untuk fine-tuning
# ADAM_LR = 1e-4 atau 2e-5 adalah nilai awal yang baik untuk fine-tuning
model.compile(optimizer=Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary() # Untuk melihat arsitektur dan jumlah parameter yang dapat dilatih

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)

# Train model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluasi model (opsional, karena EarlyStopping sudah restore_best_weights)
# loss, accuracy = model.evaluate(test_generator)
# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ input_layer_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_20       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        144 │ activation_20[0]… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_19 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        256 │ re_lu_19[0][0]    │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_add   │ (None, 112, 112,  │          0 │ activation_20[0]… │
│ (Add)               │ 16)               │            │ expanded_conv_pr… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │      1,024 │ expanded_conv_ad… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_ex… │ (None, 112, 112,  │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_20 (ReLU)     │ (None, 112, 112,  │          0 │ expanded_conv_1_… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 113, 113,  │          0 │ re_lu_20[0][0]    │
│ (ZeroPadding2D)     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        576 │ expanded_conv_1_… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_1_de… │ (None, 56, 56,    │        256 │ expanded_conv_1_… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_21 (ReLU)     │ (None, 56, 56,    │          0 │ expanded_conv_1_

 Total params: 3,002,118 (11.45 MB)

 Trainable params: 958,966 (3.66 MB)

 Non-trainable params: 2,043,152 (7.79 MB)

Epoch 1/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 85s 176ms/step - accuracy: 0.3435 - loss: 1.7027 - val_accuracy: 0.7290 - val_loss: 0.7989 - learning_rate: 2.0000e-05
Epoch 2/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 70s 159ms/step - accuracy: 0.7353 - loss: 0.7506 - val_accuracy: 0.8507 - val_loss: 0.4446 - learning_rate: 2.0000e-05
Epoch 3/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 70s 160ms/step - accuracy: 0.8220 - loss: 0.5270 - val_accuracy: 0.8890 - val_loss: 0.3297 - learning_rate: 2.0000e-05
Epoch 4/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 73s 165ms/step - accuracy: 0.8496 - loss: 0.4369 - val_accuracy: 0.9037 - val_loss: 0.2809 - learning_rate: 2.0000e-05
Epoch 5/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 71s 162ms/step - accuracy: 0.8654 - loss: 0.3740 - val_accuracy: 0.9103 - val_loss: 0.2579 - learning_rate: 2.0000e-05
Epoch 6/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 69s 158ms/step - accuracy: 0.8743 - loss: 0.3446 - val_accuracy: 0.9173 - val_loss: 0.2444 - learning_rate: 2.0000e-05
Epoch 7/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 70s 15

In [7]:
# Alternatif: Simpan model dalam format HDF5 (.h5)
model.save('model_saya_final.h5')
print("Model telah disimpan dalam format .h5")

Model telah disimpan dalam format .h5


In [8]:
# Evaluasi model (opsional, karena EarlyStopping sudah restore_best_weights)
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 40ms/step - accuracy: 0.9293 - loss: 0.1818
Test Loss: 0.18607357144355774
Test Accuracy: 0.9293333292007446


In [4]:
# Parameters
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
EPOCHS = 50 # Tingkatkan epoch, EarlyStopping akan menangani jika terlalu banyak
NUM_CLASSES = len(os.listdir('dataset/train')) # Pastikan path ini benar

# Data generators
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    width_shift_range=0.2, # Tambahan augmentasi ringan
    height_shift_range=0.2, # Tambahan augmentasi ringan
    shear_range=0.2, # Tambahan augmentasi ringan
    horizontal_flip=True,
    fill_mode='nearest' # Mode untuk mengisi piksel baru setelah augmentasi
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    'dataset/train',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'dataset/test',
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Load base model
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(224, 224, 3), include_preprocessing=False)

# --- Fine-tuning strategy ---
# Freeze sebagian besar layer awal, dan unfreeze beberapa layer terakhir
base_model.trainable = True

# Tentukan berapa banyak layer dari akhir yang ingin di-fine-tune
# Ini adalah hyperparameter, Anda mungkin perlu bereksperimen
# Mari kita coba unfreeze 20 layer terakhir dari MobileNetV3Large
# Anda bisa mengecek jumlah layer total dengan: print(len(base_model.layers))
fine_tune_at_layer_count = 20 # Unfreeze N layer terakhir
if len(base_model.layers) > fine_tune_at_layer_count:
    for layer in base_model.layers[:-fine_tune_at_layer_count]: # Bekukan semua KECUALI N terakhir
        layer.trainable = False
else: # Jika model lebih pendek dari N, unfreeze semua
    print(f"Peringatan: Model memiliki {len(base_model.layers)} layer, kurang dari {fine_tune_at_layer_count} yang diminta untuk fine-tune. Semua layer base model akan dilatih.")
    for layer in base_model.layers:
        layer.trainable = True
# --------------------------

# Add custom top layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
# Anda bisa mencoba menambah layer Dense di sini jika diperlukan
# x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x) # Sesuaikan dropout rate jika perlu (misal 0.4 atau 0.5)
predictions = Dense(NUM_CLASSES, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Compile model dengan learning rate yang lebih rendah untuk fine-tuning
# ADAM_LR = 1e-4 atau 2e-5 adalah nilai awal yang baik untuk fine-tuning
model.compile(optimizer=Adam(learning_rate=2e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary() # Untuk melihat arsitektur dan jumlah parameter yang dapat dilatih

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, min_lr=1e-7)

# Train model
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr]
)

# Evaluasi model (opsional, karena EarlyStopping sudah restore_best_weights)
# loss, accuracy = model.evaluate(test_generator)
# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv (Conv2D)       │ (None, 112, 112,  │        432 │ input_layer_1[0]… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv_bn             │ (None, 112, 112,  │         64 │ conv[0][0]        │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_18       │ (None, 112, 112,  │          0 │ conv_bn[0][0]     │
│ (Activation)        │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 113, 113,  │          0 │ activation_18[0]… │
│ (ZeroPadding2D)     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │        144 │ expanded_conv_de… │
│ (DepthwiseConv2D)   │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 56, 56,    │         64 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_14 (ReLU)     │ (None, 56, 56,    │          0 │ expanded_conv_de… │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │          0 │ re_lu_14[0][0]    │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │        136 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 8)   │          0 │ expanded_conv_sq… │
│ (ReLU)              │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 1, 1, 16)  │        144 │ expanded_conv_sq… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_9 (Add)         │ (None, 1, 1, 16)  │          0 │ expanded_conv_sq… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_15 (ReLU)     │ (None, 1, 1, 16)  │          0 │ add_9[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multiply_9          │ (None, 1, 1, 16)  │          0 │ re_lu_15[0][0]    │
│ (Multiply)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_sque… │ (None, 56, 56,    │          0 │ re_lu_14[0][0],   │
│ (Multiply)          │ 16)               │            │ multiply_9[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 56, 56,    │        256 │ expanded_conv_sq… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 942,582 (3.60 MB)

 Trainable params: 354,198 (1.35 MB)

...

  1/439 ━━━━━━━━━━━━━━━━━━━━ 1:21:43 11s/step - accuracy: 0.0625 - loss: 2.3994

I0000 00:00:1747966923.726741     621 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


439/439 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - accuracy: 0.2753 - loss: 1.9174

/home/abiyamf/anaconda3/envs/PCDL_env/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


439/439 ━━━━━━━━━━━━━━━━━━━━ 90s 180ms/step - accuracy: 0.2755 - loss: 1.9167 - val_accuracy: 0.6257 - val_loss: 1.1764 - learning_rate: 2.0000e-05
Epoch 2/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 59s 135ms/step - accuracy: 0.6342 - loss: 1.0315 - val_accuracy: 0.7933 - val_loss: 0.7069 - learning_rate: 2.0000e-05
Epoch 3/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 59s 135ms/step - accuracy: 0.7541 - loss: 0.7291 - val_accuracy: 0.8397 - val_loss: 0.4894 - learning_rate: 2.0000e-05
Epoch 4/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 59s 135ms/step - accuracy: 0.7993 - loss: 0.5779 - val_accuracy: 0.8647 - val_loss: 0.3934 - learning_rate: 2.0000e-05
Epoch 5/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 59s 135ms/step - accuracy: 0.8203 - loss: 0.5149 - val_accuracy: 0.8760 - val_loss: 0.3446 - learning_rate: 2.0000e-05
Epoch 6/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 59s 135ms/step - accuracy: 0.8341 - loss: 0.4771 - val_accuracy: 0.8810 - val_loss: 0.3179 - learning_rate: 2.0000e-05
Epoch 7/50
439/439 ━━━━━━━━━━━━━━━━━━━━ 60s 136ms/step - 

In [5]:
# Alternatif: Simpan model dalam format HDF5 (.h5)
model.save('model_say2_final.h5')
print("Model telah disimpan dalam format .h5")

Model telah disimpan dalam format .h5


In [6]:
# Evaluasi model (opsional, karena EarlyStopping sudah restore_best_weights)
loss, accuracy = model.evaluate(test_generator)
print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")

94/94 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.9205 - loss: 0.2043
Test Loss: 0.20890958607196808
Test Accuracy: 0.9256666898727417
